In [ ]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re

# Load the dataset
data = pd.read_csv('output_new.csv')

# Data Cleaning
data['team_1'] = data['team_1'].fillna('Unknown')
data['team_2'] = data['team_2'].fillna('Unknown')
data['pom'] = data['pom'].fillna('No Award')
data['best_batters'] = data['best_batters'].fillna('Unknown')
data['best_bowlers'] = data['best_bowlers'].fillna('Unknown')
data['world_cup_year'] = data['world_cup_year'].astype(str)

def extract_winner(result):
    match = re.search(r'\b([A-Za-z\s]+)\s+won\b', result)
    if match:
        return match.group(1).strip() 
    return None  

# Initialize Dash app
app = Dash(__name__)

# App Layout
app.layout = html.Div([
    html.H1("ICC Men’s Cricket World Cup - A Journey Through History",style={'fontSize':48,'color':'darkblue','textAlign': 'center'}),

    html.Div([
        dcc.Dropdown(
            id='match_category_filter',
            options=[{'label': cat, 'value': cat} for cat in data['match_category'].unique()],
            placeholder='Select Match Category',
            style={'width': '300px'}
        ),
        dcc.Dropdown(
            id='year_filter',
            options=[{'label': year, 'value': year} for year in data['world_cup_year'].unique()],
            placeholder='Select Year',
            style={'width': '300px'}
        ),
    ], style={'display': 'flex','flexDirection':'column','alignItems':'center','gap': '10px', 'marginBottom': '20px'}),

    html.Div([
        dcc.Graph(id='pie_chart_matches'),
        dcc.Graph(id='bar_chart_runs'),
        dcc.Graph(id='bar_chart_wickets'),
        dcc.Graph(id='bar_chart_mom'),
    ]),

    html.Div([
        dcc.Dropdown(
            id='team_filter',
            options=[{'label': team, 'value': team} for team in pd.concat([data['team_1'], data['team_2']]).unique()],
            placeholder='Select Teams',
            multi=True,
            style={'width': '300px', 'margin': '0 auto','marginBottom': '20px'}),
        dcc.Graph(id='line_chart_average_runs'),
    ], style={'marginTop': '50px'}),

    html.Div([
        dcc.Graph(id='pairwise_comparison_matrix'),
    ], style={'marginTop': '50px'})
])

In [ ]:
# Callback to update graphs
@app.callback(
    [
        Output('pie_chart_matches', 'figure'),
        Output('bar_chart_runs', 'figure'),
        Output('bar_chart_wickets', 'figure'),
        Output('bar_chart_mom', 'figure'),
        Output('pairwise_comparison_matrix', 'figure')
    ],
    [
        Input('match_category_filter', 'value'),
        Input('year_filter', 'value')
    ]
)
def update_charts(match_category, year):
    # Filter data based on dropdown selections
    filtered_data = data.copy()
    if match_category:
        filtered_data = filtered_data[filtered_data['match_category'] == match_category].copy()
    if year:
        filtered_data = filtered_data[filtered_data['world_cup_year'] == year].copy()

    # Pie Chart: Total Matches Won by Team
    filtered_data['winner'] = filtered_data['result'].apply(extract_winner)
    filtered_data['winner'] = filtered_data['winner'].dropna()
    matches_won = filtered_data['winner'].value_counts().reset_index()
    matches_won.columns = ['Team', 'Matches Won']
    pie_chart = px.pie(matches_won, names='Team', values='Matches Won', title='Total Matches Won by Team')
    
    # Bar Chart: Total Runs by Team
    total_runs_team_1 = filtered_data.groupby('team_1')['team_1_runs'].sum().reset_index()
    total_runs_team_2 = filtered_data.groupby('team_2')['team_2_runs'].sum().reset_index()
    total_runs_team_1.columns = ['Team', 'Runs']
    total_runs_team_2.columns = ['Team', 'Runs']
    total_runs = pd.concat([total_runs_team_1, total_runs_team_2])
    total_runs = total_runs.groupby('Team')['Runs'].sum().reset_index()
    bar_chart_runs = px.bar(total_runs, x='Team', y='Runs', title='Total Runs by Team', text='Runs')

In [ ]:
 # Line Chart: Variation of Average Runs Over the Years
    avg_runs_team_1 = filtered_data.groupby(['world_cup_year', 'team_1'])['team_1_runs'].mean().reset_index()
    avg_runs_team_2 = filtered_data.groupby(['world_cup_year', 'team_2'])['team_2_runs'].mean().reset_index()
    avg_runs_team_1.columns = ['Year', 'Team', 'Average Runs']
    avg_runs_team_2.columns = ['Year', 'Team', 'Average Runs']
    avg_runs = pd.concat([avg_runs_team_1, avg_runs_team_2])
    line_chart_average_runs = px.line(
        avg_runs,
        x='Year',
        y='Average Runs',
        color='Team',
        title='Variation of Average Runs of Each Team Over the Years',
    )
    
    # Bar Chart: Total Wickets by Team
    total_wickets_team_1 = filtered_data.groupby('team_1')['team_1_wickets'].sum().reset_index()
    total_wickets_team_2 = filtered_data.groupby('team_2')['team_2_wickets'].sum().reset_index()
    total_wickets_team_1.columns = ['Team', 'Wickets']
    total_wickets_team_2.columns = ['Team', 'Wickets']
    total_wickets = pd.concat([total_wickets_team_1, total_wickets_team_2])
    total_wickets = total_wickets.groupby('Team')['Wickets'].sum().reset_index()
    bar_chart_wickets = px.bar(total_wickets, x='Team', y='Wickets', title='Total Wickets by Team', text='Wickets')
    
    # Bar Chart: Most Valuable Player
    mom_counts = filtered_data[filtered_data['pom'] != 'No Award']['pom'].value_counts().head(10).reset_index()
    mom_counts.columns = ['Player', 'POM Awards']
    bar_chart_mom = px.bar(mom_counts, y='Player', x='POM Awards', title='Most Valuable Players (POM Awards)', text=None, orientation='h')

In [ ]:
        # Calculate the Head-to-Head (H2H) Comparison Matrix
    teams = pd.concat([filtered_data['team_1'], filtered_data['team_2']]).unique()
    h2h_matrix = pd.DataFrame(0, index=teams, columns=teams)

    for _, row in filtered_data.iterrows():
        team_1 = row['team_1']
        team_2 = row['team_2']
        if team_1 != team_2:  # Skip if the same team faces itself
            h2h_matrix.loc[team_1, team_2] += 1
            h2h_matrix.loc[team_2, team_1] += 1  # Since it's a mutual match

	# Create Table for H2H Matrix
    h2h_table = go.Figure(data=[go.Table(
        header=dict(
            values=["Teams"] + list(h2h_matrix.columns),  # Add team names to header
            fill_color='lightgrey',
            align='center',
            font=dict(size=12, color='black')
        ),
        cells=dict(
            values=[h2h_matrix.index] + [h2h_matrix[col].values for col in h2h_matrix.columns],  # Fill table with H2H data
            fill_color='white',
            align='center',
            font=dict(size=11)
        )
    )])
    h2h_table.update_layout(
        title="Head-to-Head Match Count Between Teams",
        margin=dict(l=20, r=20, t=40, b=20)
    )
    
    return pie_chart, bar_chart_runs, bar_chart_wickets, bar_chart_mom, h2h_table

In [ ]:
@app.callback(
    Output('line_chart_average_runs', 'figure'),
    [Input('team_filter', 'value')]
)
def update_line_chart(teams):
    if not teams:
        return px.line(title="Average Runs Variation by Teams")
    
    # Filter data for the selected teams
    team_runs = data[(data['team_1'].isin(teams)) | (data['team_2'].isin(teams))].copy()
    
    # Calculate total runs for each team
    team_runs['team'] = team_runs.apply(
        lambda row: row['team_1'] if row['team_1'] in teams else row['team_2'], axis=1
    )
    team_runs['total_runs'] = team_runs.apply(
        lambda row: row['team_1_runs'] if row['team_1'] in teams else row['team_2_runs'], axis=1
    )
